# Exploratory analysis speakers aggregations

In [1]:
#importing the required modules
import seaborn as sns
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import scipy.stats as stats
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
# Small adjustments to default style of plots, making sure it's readable and colorblind-friendly everywhere
plt.style.use('seaborn-colorblind')
plt.rcParams.update({'font.size' : 12.5,
                     'figure.figsize':(20,7)})

## 1. Import data


### Metadata

Pandas requires pyarrow to read parquet files, which can be installed using conda install pyarrow -c conda-forge.
You can load this file as a pandas dataframe using df = pd.read_parquet(<path_to_file>).

In [3]:
#Define the path for folder containing data
#TO BE MODIFIED ACCORDING HIS OWN FOLER containing data in local
#Jules
path = r'/Users/jules/kDrive/onedrive/Documents_Onedrive/EPFL/MA3/ADA/Project/Dataset/'
df_metadata = pd.read_parquet(path+'speaker_attributes.parquet')
#df_metadata.head()

### Quotations dataset

In [4]:
#Import the dataset sample

#Jules: 
path = r'/Users/jules/kDrive/onedrive/Documents_Onedrive/EPFL/MA3/ADA/Project/Dataset/'

df_sample = pd.read_json(path+'Sample_classified_1Mio_v1.json.bz2',compression = 'bz2',lines = True)


In [6]:
df_sample.sample(100)

quoteID                                          quotation  \
77792   2015-10-07-099301  The most sought-after CIOs are those individua...   
52035   2015-10-21-121210      This is not a job I've ever wanted or sought.   
258478  2017-03-04-023281  It's called catching the car if you're the dog...   
633579  2020-01-22-039549  I was trying to think what to do out there to ...   
502071  2019-11-02-024438  Increase in the level of pollution is a matter...   
...                   ...                                                ...   
376742  2018-10-02-090735  That will help us on a competitive front wheth...   
139141  2016-07-09-029175  It is the rust on the razor that sliced the th...   
603956  2020-03-06-000732  a different, totally different person than [ h...   
639149  2020-01-21-009229  Being Canadian, you're born into this world wi...   
22771   2015-03-21-008046  For a company as large as yours, I think it's ...   

                   speaker                                               qids  \
77792            John Reed  [Q10517212, Q28055453, Q28363859, Q528979, Q62...   
52035            Paul Ryan  [Q15610524, Q16194779, Q16196240, Q19849048, Q...   
258478        Patty Murray                                          [Q258825]   
633579  Caroline Wozniacki                                           [Q30767]   
502071     Rakesh Aggarwal                              [Q37391314, Q7286596]   
...                    ...                                                ...   
376742         Bill Haslam                                          [Q862186]   
139141        Maya Angelou                                           [Q19526]   
603956        Bill Clinton                                            [Q1124]   
639149        Larry Walker                   [Q11752886, Q22337579, Q3218116]   
22771             Jim Burt                                        [Q16193029]   

                      date  numOccurrences  \
77792  2015-10-07 10:52:00               1   
52035  2015-10-21 00:57:27               3   
258478 2017-03-04 06:18:00               1   
633579 2020-01-22 07:08:57               1   
502071 2019-11-02 02:04:44               1   
...                    ...             ...   
376742 2018-10-02 22:53:59               1   
139141 2016-07-09 22:43:00               1   
603956 2020-03-06 03:23:07               2   
639149 2020-01-21 00:00:00              37   
22771  2015-03-21 00:01:00               1   

                                                   probas  \
77792                 [[John Reed, 0.659], [None, 0.341]]   
52035               [[Paul Ryan, 0.8941], [None, 0.1059]]   
258478           [[Patty Murray, 0.8324], [None, 0.1676]]   
633579  [[Caroline Wozniacki, 0.9083], [None, 0.0831],...   
502071        [[Rakesh Aggarwal, 0.7508], [None, 0.2492]]   
...                                                   ...   
376742            [[Bill Haslam, 0.7766], [None, 0.2234]]   
139141           [[Maya Angelou, 0.6655], [None, 0.3345]]   
603956  [[Bill Clinton, 0.7039], [None, 0.1299], [Genn...   
639149  [[Larry Walker, 0.6862], [None, 0.2973], [Ferg...   
22771   [[Jim Burt, 0.6277], [None, 0.2431], [Gary Bro...   

                                                     urls phase      p1  \
77792   [http://www.cioinsight.com/it-management/caree...     E  0.6590   
52035   [http://feeds.theguardian.com/c/34708/f/663828...     E  0.8941   
258478  [https://www.washingtonpost.com/powerpost/some...     E  0.8324   
633579  [https://indianexpress.com/article/sports/tenn...     E  0.9083   
502071  [https://realty.economictimes.indiatimes.com/n...     E  0.7508   
...                                                   ...   ...     ...   
376742  [http://wkrn.com/news/after-talking-with-trump...     E  0.7766   
139141  [http://dailyprogress.com/lifestyles/alleghany...     E  0.6655   
603956  [http://honey.nine.com.au/latest/bill-clinton-...     E  0.7039   
639149  [http://www.fox13news.com/sports/derek-j

### Lookup table for QIDs

#### Using the provided csv file to link qids with meaningful names

In [8]:
#read the file 

df_lookup = pd.read_csv('wikidata_labels_descriptions_quotebank.csv.bz2',compression = 'bz2',index_col='QID')
#df_lookup.sample(20)

### 2. Merge speakers metadata with quotations 

Join quotations data with metadata :

In [9]:
#Create sub-sample
#df_sub = df_sample.sample(int(1*10e3))
df_sub = df_sample.copy()
s1 = len(df_sub)

#For the moment, choose rows which at minimum 1 qid associated
#Remove this row when using the data that have been wrangled
df_sub = df_sub[[len(a)>0 for a in df_sub['qids']]]

s2 = len(df_sub)

print('There is {} % of rows that have no qid associated with (on {} rows)'.format((1-s2/s1)*100,s1))

#Choose the first QID that is associated with (to be improved)
df_sub['qid_unique'] = df_sub['qids'].apply(lambda x: x[0]) 

There is 0.0 % of rows that have no qid associated with (on 668534 rows)


In [10]:
#Group quotation by speaker

df_grouped=df_sub.groupby('qid_unique') \
['quoteID'] \
.count() \
.reset_index(name='count') \
.sort_values(['count'], ascending=False) 

In [111]:
#Create a column to see the fraction of colloquial quotes over total quotations.

#df_grouped['colloquial_fraction'] = 



In [160]:
df1 = df_sub.groupby('qid_unique') \
['quoteID'] \
.count()\
.reset_index(name='Number of quotes')

df2 = df_sub.groupby(['qid_unique', 'colloquial']).size().unstack(fill_value=0).reset_index()


df2['quotes number'] = df2[1]+df2[0]
df2.sort_values(['quotes number'], ascending=False) 

colloquial qid_unique     0     1  quotes number
55531          Q22686  4245  1429           5674
165653            Q76  1008   359           1367
1431            Q1058   867   496           1363
89295         Q359442   723   268            991
101251        Q450675   641   271            912
...               ...   ...   ...            ...
109358      Q48225423     1     0              1
109357      Q48224900     0     1              1
47759       Q20982416     1     0              1
47760       Q20982621     1     0              1
173959        Q999889     0     1              1

[173960 rows x 4 columns]

In [161]:
df2

colloquial qid_unique  0  1  quotes number
0            Q1000053  2  1              3
1            Q1000275  1  0              1
2            Q1000290  1  3              4
3            Q1000314  1  3              4
4            Q1000366  0  1              1
...               ... .. ..            ...
173955        Q998884  1  0              1
173956        Q999299  1  0              1
173957        Q999332  2  3              5
173958        Q999577  4  3              7
173959        Q999889  0  1              1

[173960 rows x 4 columns]

In [126]:
df3.reset_index()
df3['fraction']=df3[1]/(df3[1]+df3[0])
df3.sort_values(['fraction'], ascending=False)

colloquial  0  1  fraction
qid_unique                
Q999889     0  1       1.0
Q4696730    0  1       1.0
Q1740079    0  1       1.0
Q4696700    0  1       1.0
Q1740086    0  1       1.0
...        .. ..       ...
Q45561071   1  0       0.0
Q45558354   1  0       0.0
Q17564389   1  0       0.0
Q17564409   1  0       0.0
Q50365960   1  0       0.0

[173960 rows x 3 columns]

In [11]:
#Merging with metadata

df_merged = df_grouped.merge(df_metadata,how='inner',left_on='qid_unique', right_on='id').sort_values(['count'], ascending=False)

In [ ]:
df3 = df.groupby(['key1', 'key2']).size().unstack(fill_value=0)
print (df3)
key2  one  two
key1          
a       2    1
b       1    1
c       0    1

In [12]:
#Replace label column at the first place

# shift column 'Name' to first position
first_column = df_merged.pop('label')
  
# insert column using insert(position,column_name,
# first_column) function
df_merged.insert(0, 'label', first_column)

In [13]:
df_merged.head(20)

label qid_unique  count  \
0         Barack Obama        Q76   1367   
1       Bernie Sanders    Q359442    991   
2              Francis    Q450675    912   
3      Hillary Clinton      Q6294    890   
4         Andrew Cuomo     Q11673    658   
5   Benjamin Netanyahu     Q43723    561   
6     Elizabeth Warren    Q434706    557   
7        Boris Johnson    Q180589    547   
8      Rodrigo Duterte    Q457786    546   
9         Nancy Pelosi    Q170581    519   
10         Mike Pompeo    Q473239    490   
11       Pep Guardiola    Q164038    482   
12        Rahul Gandhi     Q10218    430   
14          Mike Pence     Q24313    427   
13      Vladimir Putin      Q7747    427   
15      Lindsey Graham     Q22212    397   
16     Mitch McConnell    Q355522    390   
17       Jeremy Corbyn    Q291169    364   
18           Elon Musk    Q317521    356   
19         Theresa May    Q264766    347   

                                              aliases  \
0   [Barack Hussein Obama II, Barack Obama II, Bar...   
1                                   [Bernard Sanders]   
2   [Jorge Mario Bergoglio, Francisco, Pope Franci...   
3   [Hillary Rodham Clinton, Hillary Rodham, Hilla...   
4                                 [Andrew Mark Cuomo]   
5   [Binyamin Netanyahu, Bibi, Bibi Netanyahu, Ben...   
6   [Elizabeth Ann Warren, Senator Warren, Elisabe...   
7   [Boris, Alexander Boris de Pfeffel Johnson, Bo...   
8   [Rodrigo Roa Duterte, Rodrigo "Rody" Roa Duter...   
9   [Nancy Patricia D'Alesandro Pelosi, Nancy Patr...   
10  [Michael Richard "Mike" Pompeo, Michael Richar...   
11  [Josep Guardiola, Josep Guardiola i Sala, Jose...   
12                                               None   
14  [Michael R. Pence, Rep. Mike Pence, Michael Pe...   
13  [Vladimir Vladimirovich Putin, Putin, Presiden...   
15                              [Lindsey Olin Graham]   
16  [Addison Mitchell McConnell, Jr., Addison Mitc...   
17                            [Jeremy Bernard Corbyn]   
18       [Elon Reeve Musk, Elon R. Musk, Elon R Musk]   
19  [Theresa Mary Brasier, Prime Minister Theresa ...   

                                     date_of_birth       nationality  \
0                          [+1961-08-04T00:00:00Z]             [Q30]   
1                          [+1941-09-08T00:00:00Z]             [Q30]   
2                          [+1936-12-17T00:00:00Z]      [Q237, Q414]   
3                          [+1947-10-26T00:00:00Z]             [Q30]   
4                          [+1957-12-06T00:00:00Z]             [Q30]   
5                          [+1949-10-21T00:00:00Z]            [Q801]   
6                          [+1949-06-22T00:00:00Z]             [Q30]   
7                          [+1964-06-19T00:00:00Z]       [Q30, Q145]   
8                          [+1945-03-28T00:00:00Z]            [Q928]   
9                          [+1940-03-26T00:00:00Z]             [Q30]   
10                         [+1963-12-30T00:00:00Z]             [Q30]   
11                         [+1971-01-18T00:00:00Z]             [Q29]   
12                         [+1970-06-19T00:00:00Z]            [Q668]   
14                         [+1959-06-07T00:00:00Z]             [Q30]   
13                         [+1952-10-07T00:00:00Z]    [Q15180, Q159]   
15                         [+1955-07-09T00:00:00Z]             [Q30]   
16                         [+1942-02-20T00:00:00Z]             [Q30]   
17  [+1949-05-26T00:00:00Z, +1949-01-01T00:00:00Z]            [Q145]   
18                         [+1971-06-28T00:00:00Z]  [Q258, Q16, Q30]   
19                         [+1956-10-01T00:00:00Z]            [Q145]   

        gender   lastrevid                             ethnic_group  \
0   [Q6581097]  1395141963  [Q49085, Q6935055, Q12826303, Q6392846]   
1   [Q6581097]  1392561607                                  [Q7325]   
2   [Q6581097]  1396054797                               [Q1056744]   
3   [Q6581072]  1393846565                                     None   
4   [Q6581097]  1393197828

In [178]:
len(df_merged)

55070

## 3. Exploratory analysis

In [ ]:
### fraction colloquial for top 10 speakers

In [ ]:
# fraction per occupation

In [ ]:
# fration per country